In [1]:
import warnings

import torch

from omnisafe.common.experiment_grid import ExperimentGrid
import omnisafe

In [2]:
eg = ExperimentGrid(exp_name='BM_Safety_Lagrangian')

# Set the algorithms.
# base_policy = ['TRPO', 'PPO']
naive_lagrange_policy = ['PPOLag', 'TRPOLag']
# second_order_policy = ['CPO']

# Set the environments.
safety_envs = [
    'SafetyPointGoal1-v0',
]
eg.add('env_id', safety_envs)

# Set the device.
avaliable_gpus = list(range(torch.cuda.device_count()))
gpu_id = [0]

if gpu_id and not set(gpu_id).issubset(avaliable_gpus):
    warnings.warn('The GPU ID is not available, use CPU instead.', stacklevel=1)
    gpu_id = None

eg.add('algo', naive_lagrange_policy)
eg.add('logger_cfgs:use_wandb', [True])
eg.add('logger_cfgs:wandb_project', ['Benchmark_Safety'])
eg.add('train_cfgs:vector_env_nums', [4])
eg.add('train_cfgs:torch_threads', [8])
eg.add('algo_cfgs:steps_per_epoch', [20000])
eg.add('train_cfgs:total_steps', [20000 * 30])
eg.add('seed', [0])

In [3]:
"""Example of training a policy from exp-x config with OmniSafe."""
import os, sys
from omnisafe.typing import NamedTuple, Tuple


def train(
    exp_id: str, algo: str, env_id: str, custom_cfgs: NamedTuple
) -> Tuple[float, float, float]:
    """Train a policy from exp-x config with OmniSafe.

    Args:
        exp_id (str): Experiment ID.
        algo (str): Algorithm to train.
        env_id (str): The name of test environment.
        custom_cfgs (NamedTuple): Custom configurations.
        num_threads (int, optional): Number of threads. Defaults to 6.
    """
    terminal_log_name = 'terminal.log'
    error_log_name = 'error.log'
    if 'seed' in custom_cfgs:
        terminal_log_name = f'seed{custom_cfgs["seed"]}_{terminal_log_name}'
        error_log_name = f'seed{custom_cfgs["seed"]}_{error_log_name}'
    sys.stdout = sys.__stdout__
    sys.stderr = sys.__stderr__
    print(f'exp-x: {exp_id} is training...')
    if not os.path.exists(custom_cfgs['logger_cfgs']['log_dir']):
        os.makedirs(custom_cfgs['logger_cfgs']['log_dir'], exist_ok=True)
    # pylint: disable-next=consider-using-with
    sys.stdout = open(
        os.path.join(f'{custom_cfgs["logger_cfgs"]["log_dir"]}', terminal_log_name),
        'w',
        encoding='utf-8',
    )
    # pylint: disable-next=consider-using-with
    sys.stderr = open(
        os.path.join(f'{custom_cfgs["logger_cfgs"]["log_dir"]}', error_log_name),
        'w',
        encoding='utf-8',
    )
    agent = omnisafe.Agent(algo, env_id, custom_cfgs=custom_cfgs)
    reward, cost, ep_len = agent.learn()
    return reward, cost, ep_len

In [4]:
# total experiment num must can be divided by num_pool
# meanwhile, users should decide this value according to their machine
eg.run(train, gpu_id=gpu_id)

Save with config of experiment grid in grid_config.json

ExperimentGrid [BM_Safety_Lagrangian] runs over parameters:

 env_id                                  

[env] 

	"SafetyPointGoal1-v0"



 algo                                    

...

Warning: trajectory cut off when rollout by epoch at 1000.0 steps.

Warning: trajectory cut off when rollout by epoch at 1000.0 steps.

Expected Improvement: 0.019028395414352417 Actual: 0.018519317731261253

Accept step at i=1

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 25.17022705078125       │
│ Metrics/EpCost                 │ 51.29999923706055       │
│ Metrics/EpLen                  │ 1000.0                  │
│ Train/Epoch                    │ 29.0                    │
│ Train/Entropy                  │ 1.0377870798110962      │
│ Train/KL                       │ 0.0003244770923629403   │
│ Train/StopIter                 │ 10.0                    │
│ Train/PolicyRatio/Mean         │ 0.9994621276855469      │
│ Train/PolicyRatio/Min          │ 0.9994621276855469      │
│ Train/PolicyRatio/Max          │ 0.9994621276855469      │
│ Train/PolicyRatio/Std          │ 0.0003803612489718944   │
│ Train/LR                       │ 0.0                     │
│ Train/PolicyStd                │ 0.6939499974250793      │
│ TotalEnvSteps                  │ 600000.0                │
│ Loss/Loss_pi                   │ -0.012346208095550537   │
│ Loss/Loss_pi/Delta             │ -0.001252450980246067   │
│ Value/Adv                      │ -6.866454960174906e-09  │
│ Loss/Loss_reward_critic        │ 0.017853161320090294    │
│ Loss/Loss_reward_critic/Delta  │ -0.00024051591753959656 │
│ Value/reward                   │ 2.4103338718414307      │
│ Loss/Loss_cost_critic          │ 1.1947695016860962      │
│ Loss/Loss_cost_critic/Delta    │ 0.12281203269958496     │
│ Value/cost                     │ 5.474118709564209       │
│ Time/Total                     │ 1683.3253173828125      │
│ Time/Rollout                   │ 42.370147705078125      │
│ Time/Update                    │ 13.121404647827148      │
│ Time/Epoch                     │ 55.491554260253906      │
│ Time/FPS                       │ 360.41522216796875      │
│ Misc/Alpha                     │ 1.0518617630004883      │
│ Misc/FinalStepNorm             │ 0.2970248758792877      │
│ Misc/gradient_norm             │ 0.12116410583257675     │
│ Misc/xHx                       │ 0.018076419830322266    │
│ Misc/H_inv_g                   │ 0.2823801338672638      │
│ Misc/AcceptanceStep            │ 1.0                     │
│ Metrics/LagrangeMultiplier/Mea │ 0.851540207862854       │
│ Metrics/LagrangeMultiplier/Min │ 0.851540207862854       │
│ Metrics/LagrangeMultiplier/Max │ 0.851540207862854       │
│ Metrics/LagrangeMultiplier/Std │ 0.0                     │
└────────────────────────────────┴─────────────────────────┘

In [5]:
LOG_DIR = 'exp-x\BM_Safety_Lagrangian\SafetyPointGoal1-v0---47335d209c482a3ccbd6eb021625183261eac3afe3e5926d3ce1926b1403f582\PPOLag-{SafetyPointGoal1-v0}\seed-000-2024-04-24-02-54-58'

evaluator = omnisafe.Evaluator()
for item in os.scandir(os.path.join(LOG_DIR, 'torch_save')):
    if item.is_file() and item.name.split('.')[-1] == 'pt':
        evaluator.load_saved(
            save_dir=LOG_DIR, model_name=item.name, camera_name='track', width=256, height=256
        )
        evaluator.render(num_episodes=5)
        evaluator.evaluate(num_episodes=5)

In [6]:
LOG_DIR = 'exp-x\BM_Safety_Lagrangian\SafetyPointGoal1-v0---b139ee2e886e92ed9936723eb2b2a096596d8bd77629da1d6e785f1b3f09e6bd\TRPOLag-{SafetyPointGoal1-v0}\seed-000-2024-04-24-04-25-54'

evaluator = omnisafe.Evaluator()
for item in os.scandir(os.path.join(LOG_DIR, 'torch_save')):
    if item.is_file() and item.name.split('.')[-1] == 'pt':
        evaluator.load_saved(
            save_dir=LOG_DIR, model_name=item.name, camera_name='track', width=256, height=256
        )
        evaluator.render(num_episodes=5)
        evaluator.evaluate(num_episodes=5)